# EL-HAMDAOUI MAROUANE | IAGI | CI-2
# TP 3 : Chatbot à Règles
# Intentions + Mémoire de Contexte

## Objectifs du TP
- Implémenter un agent symbolique (sans apprentissage automatique)
- Détecter des intentions à partir de mots-clés
- Gérer une mémoire de contexte (dernier sujet abordé)
- Concevoir des modules distincts : détection, mémoire, génération de réponses
- Comprendre les limitations des systèmes à règles

## Scénario

Développer un chatbot capable de :
- Répondre aux salutations
- Donner l'heure actuelle
- Fournir une météo factice pour différentes villes
- Se présenter
- Dire au revoir

Le chatbot reconnaît des intentions (salutation, météo, heure, identité, aurevoir) et se souvient
du dernier sujet pour interpréter des questions ambiguës comme « Et demain ? » après une
question météo.

## Partie 1 : Intentions et Mots-Clés

Dans cette première partie, vous allez définir les intentions que votre chatbot peut reconnaître
et les mots-clés associés à chaque intention.

### Question 1 : Définition du dictionnaire INTENTS

Complétez le dictionnaire INTENTS avec au moins 2 synonymes ou variantes par intention.
Vous devez couvrir les intentions suivantes :

- salutation : mots pour dire bonjour
- meteo : mots liés à la météo et aux villes
- heure : mots pour demander l'heure
- identite : questions sur l'identité du bot
- aurevoir : mots pour dire au revoir

**Exemple de structure :**
```python
INTENTS = {
"salutation": ["bonjour", "salut", ...],
"meteo": ["meteo", "temps", ...],
...
}
# Votre code Python :
```

In [7]:
INTENTS = {
"salutation": ["bonjour", "salut", "hello", "hi"],
"meteo": ["meteo", "temps", "weather"],
"au_revoir": ["au revoir", "bye", "à bientôt", "see you"],
"remerciement": ["merci", "thank you", "thanks"],
"demande_information": ["informations", "info", "details", "tell me about"],
}

## Partie 2 : Normalisation du Texte

La normalisation du texte est une étape cruciale pour améliorer la robustesse de la détection
d'intentions. Elle permet de traiter uniformément les variations d'écriture.

### Question 2 : Implémentation de la fonction normalize

Implémentez la fonction normalize(text) qui doit :

1. Convertir tout le texte en minuscules
2. Retirer les accents (approximation : é→e, è→e, à→a, etc.)
3. Supprimer la ponctuation (remplacer par des espaces)
4. Supprimer les espaces multiples et les espaces en début/fin

**Exemples de tests :**
```python
normalize("Bonjour!!") →"bonjour"
normalize("QUELLE heure ?") →"quelle heure"
normalize("Météo à Casablanca") →"meteo a casablanca"
# Votre code Python :
```

In [8]:
import re
import unicodedata

def normalize(text):
    if not isinstance(text, str):
        text = str(text)

    s = text.lower()
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

# Test cases
print(normalize("Bonjour!!"))  # Expected: "bonjour"
print(normalize("QUELLE heure ?"))  # Expected: "quelle heure"
print(normalize("Météo à Casablanca"))  # Expected: "meteo a casablanca"

bonjour
quelle heure
meteo a casablanca


## Partie 3 : Détection d'Intention

Maintenant que vous avez défini les intentions et la normalisation, vous allez implémenter la
fonction qui détecte l'intention d'un message utilisateur.

### Question 3 : Implémentation de detect_intent

Implémentez la fonction detect_intent(message) qui :

1. Normalise le message avec la fonction normalize
2. Parcourt le dictionnaire INTENTS
3. Pour chaque intention, teste si un mot-clé apparaît dans le message
4. Retourne le nom de l'intention trouvée, ou "inconnu" si aucune correspondance

**Exemples de tests :**

```python
detect_intent("Salut") →"salutation"
detect_intent("Quelle est la meteo de Rabat ?") →"meteo"
detect_intent("Quelle heure svp") →"heure"
detect_intent("Qui es-tu ?") →"identite"
# Votre code Python :
```

In [9]:
def detect_intent(message):
    msg = normalize(message)
    for intent, keywords in INTENTS.items():
        for kw in keywords:
            if normalize(kw) in msg:
                return intent
    return "inconnu"

print(detect_intent("Salut"))# Expected: "salutation"
print(detect_intent("Quelle est la meteo de Rabat ?")) # Expected: "meteo"
print(detect_intent("Quelle heure svp")) # Expected: "inconnu"
print(detect_intent("Qui es-tu ?")) # Expected: "inconnu"

salutation
meteo
inconnu
inconnu


## Partie 4 : Génération de Réponses + Mémoire

C'est la partie la plus importante du TP ! Vous allez implémenter la fonction qui génère les
réponses en fonction de l'intention détectée et qui gère la mémoire de contexte.

### Question 4 : Implémentation de respond avec mémoire

Implémentez la fonction respond(intent, message, memory) qui doit :

#### A. Réponses de base
- salutation : Retourner une salutation aléatoire parmi plusieurs options
- heure : Retourner l'heure actuelle (format HH:MM)
- identite : Se présenter comme un agent IA
- aurevoir : Dire au revoir

#### B. Météo avec extraction de ville

Pour l'intention "meteo" :

5. Extraire la ville mentionnée dans le message (casablanca, rabat, marrakech)
6. Si aucune ville n'est mentionnée, utiliser "casablanca" par défaut
7. Mémoriser last_intent = "meteo" et last_city = ville
8. Retourner la météo pour cette ville

**Données météo factices :**

`FAKE_WEATHER = {"casablanca": 23, "rabat": 22, "marrakech": 26}`

#### C. Gestion de la mémoire

Si l'intention est "inconnu" ET que last_intent dans la mémoire vaut "meteo" :

- Vérifier si le message contient des mots comme "demain", "apres demain", "ensuite"
- Si oui, utiliser last_city de la mémoire pour donner la prévision

**Exemple de dialogue avec mémoire :**

```
Vous: La meteo a Marrakech svp
Bot: Météo à Marrakech : 26°C
Vous: Et demain ?
Bot: Prévision à Marrakech demain : ensoleillé, ~26°C
Votre code Python (fonction respond complète) :
```

In [10]:
import random
from datetime import datetime

FAKE_WEATHER = {"casablanca": 23, "rabat": 22, "marrakech": 26}

def respond(intent, message, memory):
    msg = normalize(message)

    if intent == "salutation":
        return random.choice(["Bonjour !", "Salut !", "Coucou !", "Bonjour, comment puis-je aider ?"])
    if intent == "heure":
        return f"Heure actuelle : {datetime.now().strftime('%H:%M')}"
    if intent == "identite":
        return "Je suis un agent conversationnel (IA) conçu pour ce TP."
    if intent == "aurevoir":
        return "Au revoir ! À bientôt."
    if intent == "meteo":
        city = None
        
        for c in FAKE_WEATHER.keys():
            if c in msg:
                city = c
                break
        
        if city is None:
            city = "casablanca"
        
        memory['last_intent'] = 'meteo'
        memory['last_city'] = city
        temp = FAKE_WEATHER.get(city, '?')
        
        return f"Météo à {city.capitalize()} : {temp}°C"

    if memory.get('last_intent') == 'meteo':
        if any(kw in msg for kw in ["demain", "apres demain", "apres-demain", "ensuite"]):
            city = memory.get('last_city', 'casablanca')
            temp = FAKE_WEATHER.get(city, '?')
            
            return f"Prévision à {city.capitalize()} demain : ensoleillé, ~{temp}°C"

    return "Désolé, je n'ai pas compris. Peux-tu reformuler ?"

memory = {}
print(respond("meteo", "La meteo a Marrakech svp", memory))
print(respond("inconnu", "Et demain ?", memory))

Météo à Marrakech : 26°C
Prévision à Marrakech demain : ensoleillé, ~26°C


## Partie 5 : Boucle de Dialogue Interactive

Maintenant que toutes les fonctions sont implémentées, vous allez créer une boucle interactive permettant de discuter avec le chatbot.

### Question 5 : Implémentation de chat_loop

Implémentez la fonction chat_loop() qui :

1. Initialise un dictionnaire memory vide
2. Affiche un message de bienvenue
3. Entre dans une boucle infinie qui lit les messages de l'utilisateur
4. Détecte l'intention du message
5. Génère et affiche la réponse
6. Sort de la boucle si l'utilisateur tape 'exit', 'quit' ou 'bye'

```python
# Votre code Python :
```

Testez votre chatbot avec le dialogue suivant :

```
Vous: Bonjour
Vous: Quelle heure est-il ?
Vous: La météo à Casablanca
Vous: Et demain ?
Vous: Qui es-tu ?
Vous: Au revoir
```

In [11]:
def chat_loop():
    memory = {}
    
    print("Bienvenue (tapez 'exit', 'quit' ou 'bye' pour quitter)")
    
    while True:
        try:
            user_input = input("> ")
        except (EOFError, KeyboardInterrupt):
            print("\nAu revoir.")
            break
        if user_input is None:
            continue
        norm = normalize(user_input)
        if norm in {"exit", "quit", "bye", "Au revoir"}:
            print("Au revoir !")
            break
        intent = detect_intent(user_input)
        reply = respond(intent, user_input, memory)
        print(reply)

if __name__ == "__main__":
    chat_loop()

Bienvenue (tapez 'exit', 'quit' ou 'bye' pour quitter)
Bonjour, comment puis-je aider ?
Désolé, je n'ai pas compris. Peux-tu reformuler ?
Météo à Casablanca : 23°C
Prévision à Casablanca demain : ensoleillé, ~23°C
Prévision à Casablanca demain : ensoleillé, ~23°C
Désolé, je n'ai pas compris. Peux-tu reformuler ?
Désolé, je n'ai pas compris. Peux-tu reformuler ?
Désolé, je n'ai pas compris. Peux-tu reformuler ?
Au revoir !


## Partie 6 : Questions de Réflexion

Cette dernière partie vous invite à réfléchir sur les concepts abordés et les limitations de l'approche par règles.

### Question 6.1 : Différence intention vs réponse

Expliquez la différence entre une intention et une réponse dans le contexte d'un chatbot. Pourquoi sépare-t-on ces deux concepts ?

**Réponse :** Dans le contexte d'un chatbot, une intention (ou intent) représente ce que l'utilisateur souhaite accomplir ou exprimer à travers son message. C'est l'objectif sous-jacent de la communication. En revanche, la réponse est ce que le chatbot renvoie à l'utilisateur en fonction de l'intention détectée. La séparation de ces deux concepts permet de mieux structurer le traitement du langage naturel et d'adapter les réponses en fonction des différentes intentions possibles.

### Question 6.2 : Utilité de la normalisation

Pourquoi la normalisation du texte aide-t-elle à améliorer la détection d'intentions ? Donnez un exemple concret où la normalisation fait la différence entre détecter ou non une intention.

**Réponse :** La normalisation du texte permet de réduire les variations dans la manière dont les utilisateurs peuvent formuler leurs messages. En convertissant le texte en minuscules, en supprimant les accents et la ponctuation, et en éliminant les espaces inutiles, on facilite la correspondance avec les mots-clés définis dans le dictionnaire d'intentions. Par exemple, sans normalisation, un message comme "Bonjour!!" pourrait ne pas être reconnu comme une salutation en raison des majuscules et de la ponctuation. Après normalisation, il devient "bonjour", ce qui correspond directement au mot-clé défini pour l'intention "salutation".

### Question 6.3 : Exemple d'utilisation de la mémoire

Donnez un exemple de dialogue où la mémoire de contexte change la réponse du chatbot. Expliquez ce qui est mémorisé et comment cela affecte la réponse.

**Réponse :** Un exemple de dialogue où la mémoire de contexte est utilisée pourrait être le suivant :

```Vous: La meteo a Marrakech svp
Bot: Météo à Marrakech : 26°C
Vous: Et demain ?
Bot: Prévision à Marrakech demain : ensoleillé, ~26°C
```

### Question 6.4 : Limites de l'approche par règles

Identifiez deux limites majeures de l'approche par règles et mots-clés pour créer un chatbot.Pour chaque limite, proposez une amélioration possible (sans nécessairement l'implémenter).

**Réponse :**

```
Limite 1 : La rigidité des règles
Amélioration proposée : Utiliser des modèles de langage basés sur l'apprentissage automatique pour permettre une compréhension plus flexible et contextuelle des requêtes des utilisateurs.

Limite 2 : La couverture limitée des intentions
Amélioration proposée : Élargir le dictionnaire d'intentions en ajoutant des synonymes et des formulations alternatives pour mieux capturer la diversité des requêtes des utilisateurs.
```

### Question 6.5 : Extension - Intention "blague"

Ajoutez une nouvelle intention "blague" qui répond par une blague aléatoire parmi 3 options. Montrez le code et testez-le avec des exemples.

```
Code de l'extension (ajouts dans INTENTS et respond) :
Exemples de test :
```

In [ ]:
INTENTS["blague"] = ["blague", "joke", "raconte", "histoire drole", "humour", "rigole"]

BLAGUES = [
    "Pourquoi les plongeurs plongent-ils toujours en arrière ? Parce que sinon ils tombent dans le bateau !",
    "Qu'est-ce qu'un crocodile qui surveille la pharmacie ? Un Lacoste garde !",
    "Pourquoi les poissons n'aiment pas jouer au tennis ? Parce qu'ils ont peur du filet !"
]


def respond(intent, message, memory):
    msg = normalize(message)

    if intent == "salutation":
        return random.choice(["Bonjour !", "Salut !", "Coucou !", "Bonjour, comment puis-je aider ?"])
    
    if intent == "heure":
        return f"Heure actuelle : {datetime.now().strftime('%H:%M')}"
    
    if intent == "identite":
        return "Je suis un agent conversationnel (IA) conçu pour ce TP."
    
    if intent == "aurevoir":
        return "Au revoir ! À bientôt."
    , mem
    if intent == "blague":
        return random.choice(BLAGUES)
    
    if intent == "meteo":
        city = None
        
        for c in FAKE_WEATHER.keys():
            if c in msg:
                city = c
                break
        
        if city is None:
            city = "casablanca"
        
        memory['last_intent'] = 'meteo'
        memory['last_city'] = city
        temp = FAKE_WEATHER.get(city, '?')
        
        return f"Météo à {city.capitalize()} : {temp}°C"

    if memory.get('last_intent') == 'meteo':
        if any(kw in msg for kw in ["demain", "apres demain", "apres-demain", "ensuite"]):
            city = memory.get('last_city', 'casablanca')
            temp = FAKE_WEATHER.get(city, '?')
            
            return f"Prévision à {city.capitalize()} demain : ensoleillé, ~{temp}°C"

    return "Désolé, je n'ai pas compris. Peux-tu reformuler ?"


print("=== Tests de l'intention 'blague' ===\n")
test_memory = {}

print("Test 1:")
print(f"User: Raconte moi une blague")
print(f"Bot: {respond(detect_intent('Raconte moi une blague'), 'Raconte moi une blague', test_memory)}\n")

print("Test 2:")
print(f"User: Tu connais une histoire drole ?")
print(f"Bot: {respond(detect_intent('Tu connais une histoire drole ?'), 'Tu connais une histoire drole ?', test_memory)}\n")

=== Tests de l'intention 'blague' ===

Test 1:
User: Raconte moi une blague
Bot: Pourquoi les poissons n'aiment pas jouer au tennis ? Parce qu'ils ont peur du filet !

Test 2:
User: Tu connais une histoire drole ?
Bot: Bonjour, comment puis-je aider ?

